# Практическое задание 7. Разработка рекомендательной системы на Python

**Инструкция:**

- Шаг 1.  Изучите материалы лекционных и практических занятий по теме 11.3. 
- Шаг 2. Постройте рекомендательную систему на примере данных о покупках. Исходные файлы: recommend_1.csv, trx_data.csv 
   * **recommend_1.csv** список из 1000 идентификаторов клиентов, рекомендуемых в качестве выходных данных.
   * **trx_data.csv** пользовательские транзакции

- Шаг 3. Реализуйте коллаборативную фильтрацию данных на основе пользователей. Используйте модель kNN. Проверить модель на покупателях с customer_id = 4 и customer_id = 21.
- Шаг 4. Опубликуйте файл расширения ipynb на платформе Odin.

У сети продуктовых магазинов разрабатывается новое мобильное приложение, позволяющее покупателям размещать заказы еще до того, как они зайдут в магазин.

В приложении должна быть возможность показывать рекомендации: когда покупатель впервые нажимает на страницу «заказ», мы можем порекомендовать добавить в его корзину 10 лучших товаров, например, одноразовую посуду, свежее мясо, чипсы и т. д.

**Цель работы: получить список рекомендаций для указанного пользователя**, например:

Входные данные: идентификатор клиента

Результат: ранжированный список товаров (идентификаторов продуктов), которые пользователь, скорее всего, захочет положить в свою (пустую) «корзину».

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings('ignore')

In [2]:
customers = pd.read_csv('recommend_1.csv')
transactions = pd.read_csv('trx_data.csv')

# Приведем данные покупок для каждого пользователя в надлежащий вид

In [3]:
transactions['products'] = transactions['products'].str.replace('|',',')
transactions = transactions.groupby(['customerId'])['products'].apply(','.join).reset_index()
transactions

,customerId,products
0,0,"20,216,52,260,93,93,93,69,69,1,1,31,31,260,256..."
1,1,"2,2,23,68,68,111,29,86,107,152,2,2,151,61,86,8..."
2,2,"111,107,29,11,11,11,33,23"
3,3,"164,227,196,196,196,196,196,196,121,227,55,100,25"
4,4,"199,289,196,15,15,200,143,270"
...,...,...
24424,28593,"130,119,119,53,53"
24425,28596,"25,15,255,20,10,211,211,211"
24426,28598,212
24427,28604,282


# Посчитаем сколько и какие товары купили пользователи

In [4]:
matrix = {}
for i in range(0, len(transactions)):
    products = transactions.products[i].split(',')
    matrix[transactions['customerId'][i]] = {}
    for j in transactions.products[i].split(','):
        if j in matrix[transactions['customerId'][i]].keys():
            matrix[transactions['customerId'][i]][j] += 1                   
        else:
            matrix[transactions['customerId'][i]].update({j : 1})            

In [5]:
df = pd.concat({k: pd.Series(v) for k, v in matrix.items()}).reset_index()  
df.columns = ['user_id', 'product_id','quantity']
df['product_id'].astype(np.int64)
df

,user_id,product_id,quantity
0,0,20,1
1,0,216,1
2,0,52,1
3,0,260,5
4,0,93,3
...,...,...,...
133580,28596,10,1
133581,28596,211,3
133582,28598,212,1
133583,28604,282,1


# Обучение модели

In [6]:
df_pivot = df.pivot(index = 'product_id', columns = 'user_id', values = 'quantity').fillna(0)
df_matrix = csr_matrix(df_pivot.values)

In [7]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

# Получение рекомендаций для пользователя

In [8]:
def find_recommendations(user_id):
    print('Recommendation for user ', user_id)
    distances, indices = model_knn.kneighbors(df_pivot.iloc[user_id,:].values.reshape(1, -1), n_neighbors = 6)
    rec_users = []
    for i in range(1, len(indices.flatten())):
        rec_users.append(indices.flatten()[i])
    df1 = df.loc[df['user_id'].isin(rec_users)].groupby('product_id').sum().sort_values(by=['quantity'], ascending=False)[:10].index.values
    print(df1)
    

In [9]:
find_recommendations(4)

Recommendation for user  4
['241' '2' '231' '6' '0' '151' '10' '86' '128' '72']


In [10]:
find_recommendations(21)

Recommendation for user  21
['273' '34' '231' '2' '11' '112' '172' '27' '38' '42']
